# Validação Cruzada

Divisao em treino e teste – base de dados  iris:


In [146]:
from sklearn.datasets import load_iris# type: ignore
iris = load_iris()

Carregar função separadora dos dados em treino e teste: 


In [147]:
from sklearn.model_selection import train_test_split# type: ignore
X1, X2, t1, t2 = train_test_split(iris.data,iris.target,test_size=0.3, stratify=iris.target)

Os dados de treino e respectivas classes estao nas variaveis X1 e t1. Os dados de teste estao
nas variaveis  X2 e t2. O parametro  test size=0.3 atribui 30% dos exemplos ao conjunto de
teste. Para os conjuntos de treino e de teste terem a mesma distribuição de classes que todos 
os dados, chamar função com parametro: stratify=iris.target

Carregar classificador (discriminante logıstico), treinar e classificar:


In [148]:
from sklearn.linear_model import LogisticRegression# type: ignore
logreg = LogisticRegression().fit(X1,t1)
print('Prob Acertos:  %.1f'%(logreg.score(X2,t2)*100))


Prob Acertos:  97.8


Probabilidade de acertos varia entre 80% e 100% – depende dos exemplos escolhidos para o
teste pela função train test split() - inicializada com o parametro random state.

# Validação cruzada K-Fold

A maneira mais directa usar validação cruzada e chamar a função
cross_val_score(). Esta recebe como parametros de entrada o 
classificador, os dados, as respectivas classes, e o numero de folds

In [149]:
from sklearn.model_selection import cross_val_score# type: ignore
from sklearn.linear_model import LogisticRegression# type: ignore
logreg = LogisticRegression(max_iter=1000)
scores = cross_val_score(logreg,iris.data,iris.target,cv=5)


In [150]:
print(scores)

[0.96666667 1.         0.93333333 0.96666667 1.        ]


A função devolve a probabilidade de acertos em cada fold de teste. Porem não se sabe as
predições individuais. Para saber as classificações atribuidas usar a função
cross_val_predict(). Esta retorna num numpy array os resultados das classes preditas
para os elementos de todo o conjunto

In [151]:
from sklearn.model_selection import cross_val_predict # type: ignore
results = cross_val_predict(logreg,iris.data,iris.target,cv=5)
print(results)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


Para ter mais controlo de como os dados sao divididos em folds, pode-se usar o  “K-fold splitter”.


In [152]:
from sklearn.model_selection import KFold # type: ignore
Kfold = KFold()

A instancia  kfold e passadaas funções cross_val_score() e cross_val_predict()
(parametro  cv=kfold)
ATENÇÃO : se nao especificado, a divisão e feita sequencialmente 
--> pode ser problematico para dados que estejam ordenados por classe:


In [153]:
Kfold = KFold(n_splits=4)
cross_val_score(logreg,iris.data,iris.target,cv=Kfold)

array([1.        , 0.94736842, 0.91891892, 0.75675676])

Ao dividir em 3 cada parte fica com uma das classes ou seja nunca vai acertar por não existir variedade


In [154]:
Kfold = KFold(n_splits=3)
cross_val_score(logreg,iris.data,iris.target,cv=Kfold)

array([0., 0., 0.])

Neste já há variedade portanto acerta por haver shuffle

In [155]:
Kfold = KFold(n_splits=3,shuffle=True,random_state=0)
cross_val_score(logreg,iris.data,iris.target,cv=Kfold)

array([0.98, 0.96, 0.96])

# Stratified KFold

Este é outro “K-fold splitter” que divide os dados em K folds.
▸ A proporção do número de exemplos por classe na base de dados ´ e mantida 
(aproximadamente) em cada fold.
▸ Muito util para dados que com diferentes percentagens de exemplos por classe, 
como e geralmente o caso em situações de detecção onde o número de exemplos 
negativos e significativamente superior aos exemplos positivos

In [156]:
from sklearn.model_selection import StratifiedKFold # type: ignore
kfold=StratifiedKFold(n_splits=3)
cross_val_score(logreg, iris.data, iris.target, cv=kfold)

array([0.98, 0.96, 0.98])

# ShuffleSplit

ShuffleSplit e outra forma de aplicar a validação cruzada. Os dados são divididos de forma 
aleatoria em dois, parte para o treino e outra para o teste, e este processo ´ e repetido K vezes. ´
Note que pode-se so usar parte dos dados para treino e teste (as percentagens relativas aos ´
dados de treino e teste nao têm que somar 100%).

In [157]:
from sklearn.model_selection import ShuffleSplit # type: ignore
kfold=ShuffleSplit(n_splits=5,train_size=0.5,test_size=0.3)
cross_val_score(logreg, iris.data, iris.target, cv=kfold)


array([0.88888889, 0.97777778, 0.97777778, 0.97777778, 0.91111111])

# StratifiedShuffleSplit

StratifiedShuffleSplit e em tudo semelhante à função ShuffleSplit, excepto que as
probabilidades a priori das classes sao mantidas nos conjuntos de treino e teste.

In [158]:
from sklearn.model_selection import StratifiedShuffleSplit # type: ignore

kfold=StratifiedShuffleSplit(n_splits=5,train_size=0.5,test_size=0.3)
cross_val_score(logreg, iris.data, iris.target, cv=kfold)

array([0.91111111, 0.93333333, 0.95555556, 0.93333333, 0.97777778])

# Matriz de Confusão

A matriz de confusão é obtida com a função confusion_matrix() do
sub-modulo sklearn.metrics.

In [194]:
from sklearn.datasets import load_iris # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.metrics import confusion_matrix # type: ignore
from sklearn.naive_bayes import GaussianNB # type: ignore
iris = load_iris()
X1,X2,t1,t2 = train_test_split(iris.data,iris.target,test_size=1/3,stratify=iris.target)
NBgau = GaussianNB().fit(X1,t1)
#t2 os que pertencem a cada classe
#se results for igual a t2 então acertou em tudo
results=NBgau.predict(X2)
print('Matriz de confusão:\n ',confusion_matrix(t2,results))
print('Proabilidade de acertos: ',NBgau.score(X2,t2))


Matriz de confusão:
  [[17  0  0]
 [ 0 17  0]
 [ 0  0 16]]
Proabilidade de acertos:  1.0


In [198]:
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold,cross_val_score,cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
import numpy as np
iris=load_iris()
kfold=StratifiedKFold(n_splits=3)
NBgau=GaussianNB()
scores=cross_val_score(NBgau,iris.data,iris.target,cv=kfold)
results=cross_val_predict(NBgau,iris.data,iris.target,cv=kfold)
print('Matriz de confusão:\n ',confusion_matrix(iris.target,results))
print('Proabilidade de acertos: ',np.mean(scores))


Matriz de confusão:
  [[50  0  0]
 [ 0 45  5]
 [ 0  4 46]]
Proabilidade de acertos:  0.94
